In [0]:
import subprocess
# Install Poppler using apt-get (for Debian-based distributions)
subprocess.run(["sudo", "apt-get", "update"])
subprocess.run(["sudo", "apt-get", "install", "-y", "poppler-utils"])

In [0]:
%run "EKIH/Includes/Configurations"

In [0]:
%run "EKIH/Includes/DBUtils"

In [0]:
import re 
import magic
import fitz
import io
from io import BytesIO, StringIO
import traceback
import hashlib 
import magic
import aspose.words as aw
#import aspose.slides as slides
from azure.storage.filedatalake import DataLakeFileClient, ContentSettings, DataLakeServiceClient
import aspose.pydrawing as drawing
import subprocess
import PyPDF2
from PyPDF2 import PdfReader
import pdf2image
from pdf2image import convert_from_bytes
import sys
import pandas as pd
from PIL import Image
import datetime
from datetime import date
from datetime import datetime, timedelta

In [0]:
dbutils.widgets.text("OPU","")
opu = dbutils.widgets.get("OPU")

dbutils.widgets.text("Source","")
source = dbutils.widgets.get("Source")

dbutils.widgets.text("Doc_Type","")
docType = dbutils.widgets.get("Doc_Type")

dbutils.widgets.text("Pipeline_RunID","")
pipelineRunID = dbutils.widgets.get("Pipeline_RunID")

In [0]:
##Oscar's detecting of Scan/Generated Documents

def PDFPageCheck(raw_content):
    doc = fitz.open(stream=raw_content,filetype='pdf')
    doc_len = len(doc)

    if doc_len > 5:
        last = 5
    else:
        last = doc_len

    image_width=[]
    image_height=[]
    result = []
    img_count_pdf = 0
    #get the size of image inside a page
    for current_page_index in range(last):
        page_img_count = 0
        for _ ,img in enumerate(doc.get_page_images(current_page_index)):
            xref = img[0]
            image = fitz.Pixmap(doc, xref)
            if image.n >= 4:   #5                  
                image = fitz.Pixmap(fitz.csRGB, image)
            image_bytes = image.tobytes()
            image = Image.open(io.BytesIO(image_bytes))
            image_width.append(image.width)
            image_height.append(image.height)
            page_img_count+=1
            img_count_pdf+=1
            del image_bytes
            
        if page_img_count > 1:
            result.append("This page may be a generated PDF.")
            ans = 'yes'

    if img_count_pdf < last:
        result.append("This page may be a generated PDF.")
        ans = 'yes'

    #if no more than 1 images extracted in a page then will proceed to this
    if len(result) > 0:
        ans_gen = isGeneratedPDF(result,doc_len,ans)
    else:
        #convert the pdf pages into images then get size of the img
        images = convert_from_bytes(raw_content, first_page=1, last_page=last)
        page_width=[]
        page_height=[]
        for i, image in enumerate(images):
            image_bytes = BytesIO()
            image.save(image_bytes, format="JPEG")
            page_width.append(image.width)
            page_height.append(image.height)
            del image_bytes  

        for i in range (last):
            answer = (is_plus_minus_2(image_width[i],page_width[i],image_height[i],page_height[i]))
            result.append(answer)

        ans = 'no'
        #ans_gen = isGeneratedPDF(result,len(doc),ans)
    return result,doc_len,ans

def getCaption(fullText):  ##To get image caption, 'Fig or Figure'
    captionText = fullText.splitlines()
    running_num = 1
    caption=[]
    for row in captionText:
        if('Figure' in row[:6] or 'Fig.' in row[:4]):
            caption.append(str(running_num)+'. : '+row)
            running_num += 1
    return caption

def getPageText(pdf_file, index):
    caption=[]
    # zoom = 1.2 # scale your pdf file by 120%
    # mat = fitz.Matrix(zoom, zoom)
 
    page = pdf_file[index]
    text = page.get_text()
    caption = getCaption(text)
    return caption

def isWholeNumber(NumCaption,NumImage): ##Image caption calculation, only 1 or 2 image with captions will be saved
    if(NumImage % NumCaption ==0):
        result = ('It is a whole number. - '+ str(NumImage % NumCaption))
        return True
    else:
        result = ('It not a whole number, having a remainder of. - '+ str(NumImage % NumCaption))
        return False

# Function to maintain FileMetadata table (Update Image caption DB - vincent code dun have this)
def InsertUpdateFileMetadata_ImageCaption(config_dict, cursor):
    # Query to get count of existing rows in FileMetadata table 
    row = cursor.execute("SELECT count(*), max(Id) as count FROM [dbo].[FileMetadata_ImageCaption] WHERE FileMetadataId = ?", config_dict['FileMetadataId']).fetchone()
       
    rows_returned = row[1]
    if (rows_returned is None): 
        # Insert new record 
        cursor.execute("INSERT INTO dbo.FileMetadata_ImageCaption(FileMetadataId,opu,pageNumber,blobPath,caption,createddate,isDeleted) values(?,?,?,?,?,getdate(),?);", config_dict['FileMetadataId'], config_dict['opu'], config_dict['pageNum'], config_dict['blobPath'], config_dict['caption'], config_dict['isDeleted'])
        JDBCconnection.commit() 
        # row = cursor.execute("SELECT IDENT_CURRENT('[dbo].[FileMetadata]');").fetchone()
        return str(rows_returned)
    else:        
        # Update existing record
        cursor.execute("UPDATE [dbo].[FileMetadata_ImageCaption] SET opu = ?, pageNumber = ? ,blobPath = ?, caption = ?,isDeleted = ?, ModifiedDate = getdate() WHERE FileMetadataId = ?;", config_dict['opu'],config_dict['pageNum'], config_dict['blobPath'], config_dict['caption'], config_dict['isDeleted'], config_dict['FileMetadataId'])
        JDBCconnection.commit() 
        return (str(rows_returned))


def updateCaption(caption,img_count,imagelist):
    cursor = JDBCconnection.cursor()
    isWholeNumberResult = isWholeNumber(len(caption),img_count)
    if(isWholeNumberResult==True):
        image_df=pd.DataFrame(imagelist,columns=['FileMetadataId','sourceTable','pageNum','blobPath','isDeleted'])
        #convert caption list into dataframe
        caption_df = pd.DataFrame(caption,columns=['caption'])
        #add a key column in caption and image datafrme 
        caption_df['key']=1
        image_df['key']=1
        #check the count number is 2 or 0, 2 is image count > caption count, 0 is image count = caption count
        count = int(len(imagelist)/len(caption))
        print('count: '+ str(count))

        if(count==1 or count == 2):
            # replicate the caption to match with image count
            num_replications = int(len(image_df)/len(caption)-1)
            for _ in range(num_replications):
                caption_df = pd.concat([caption_df,caption_df], ignore_index=True)
            #set accending order for the caption
            caption_df = caption_df.sort_values('caption', ascending=True)

            # set the sequance key value
            key = 1
            new_df=[]
            for index, i in image_df.iterrows():
                i['key'] = key
                new_df.append(i)
                key += 1
            image_df = pd.DataFrame(new_df)

            key = 1
            new_df=[]
            for index, i in caption_df.iterrows():
                i['key'] = key
                new_df.append(i)
                key += 1
            caption_df = pd.DataFrame(new_df)
            #merge the caption and image dataframe
            final_df = image_df.merge(caption_df, on=['key'])
            final_df = final_df.drop('key',axis=1)

            #insert the caption into FileMetadata_ImageCaption
            for index,i in final_df.iterrows():
                captionconfig_dict={
                    'FileMetadataId':i['FileMetadataId'],
                    'blobPath':i['blobPath'],
                    'pageNum':i['pageNum'],
                    'caption':i['caption'],
                    'isDeleted':i['isDeleted'],
                    'opu':i['sourceTable']
                }
                print(captionconfig_dict)
                
                InsertUpdateFileMetadata_ImageCaption(captionconfig_dict,cursor)
                print('Updated db successfully')
                #JDBCconnection.commit() 
    else:
        print('is not whole number')

def isGeneratedPDF(result,numpage,ans):
    #check the 5 pages result 
    count = 0
    if ans == 'yes':
        return True
    for i in result:
        if(i == "This page may be a generated PDF."):
            count += 1
    if(count >= 3 and numpage>=5):
        return True 
    elif(count >= 2 and numpage==4):
        return True
    elif(count >= 1 and numpage==3):
        return True
    elif(count == 1 and numpage==2):
        return True
    else:
        return False

def is_plus_minus_2(n1, n2, n3, n4):
    #size = page - image
    #page - image, if negative then is scanned doc
    width = n2 - n1
    height = n4 - n3

    if width <= 0 and height <= 0:
        result="This page may be a scanned PDF."
    elif width > 400 and height > 400:
         result="This page may be a scanned PDF."
    else:
        result="This page may be a generated PDF."
    return result

#Incremental code
def is_incremental(cursor,opu,docType):
    source_str = source.split('_')[0]
    container_schema = 'EKIH'
    
    query = "SELECT Is_Incremental FROM [dbo].[ingestion_enigma_config_image_extraction] WHERE Source_Table_Name = '{}' AND Doc_Type = '{}' and Source = '{}'".format(opu,docType.upper(),source_str)
    result = cursor.execute(query).fetchone()
    
    if result is None:
        inc_val = '0'

        cursor.execute("INSERT INTO [dbo].[ingestion_enigma_config_image_extraction] (Is_Incremental,Container_Schema,Source_Table_Name,Is_active,Source,Doc_Type) values(?,?,?,'1',?,?);",inc_val,container_schema,opu,source_str,docType.upper())
        JDBCconnection.commit()
        return inc_val 
    else:
        return result[0]

def latest_runtime(cursor,opu):
    
    execution_end_time_query = "SELECT Top(1) ExecutionEndTime FROM [dbo].[ingestion_log_image_extraction] WHERE OPU = '{}' and ExecutionType = 'Image Extraction' ORDER BY ExecutionStartTime DESC".format(opu)

    execution_end_time = cursor.execute(execution_end_time_query).fetchone()

    return execution_end_time[0]

def update_incremental(cursor,opu,docType):
    
    cursor.execute("UPDATE [dbo].[ingestion_enigma_config_image_extraction] set Is_Incremental='1' where Source_Table_Name='{}' AND Doc_Type = '{}'".format(opu, docType.upper()))
    JDBCconnection.commit()
        
    print('\nUpdated Incremental DB successfully')


def get_blob(container, path, docType, docFormat):
    service_client = DataLakeServiceClient(account_url = "https://{0}.dfs.core.windows.net/".format(storage_acc), credential=token_credential)
    file_system_client = service_client.get_file_system_client(file_system=container) 
    paths = file_system_client.get_paths(path)   
    files = [i.name for i in paths]
    file_list = []
    for i in files:
        if docType.upper() in i.upper() and docFormat.upper() in i.upper():
            file_list.append(i)
    return file_list

def path_validation(cursor, folder_paths, source, opu, docType, docFormat):

    if source.rsplit('_', 1)[0] != 'PEDMS':
        if source.rsplit('_', 1)[0] == 'ECMS':#FOR SKILL and ECMS
            new_opu = docType
            docType = ''
        else:
            new_opu=''

        file_id_query = "SELECT ID FROM [dbo].[FileMetadata] WHERE Source LIKE '%{}%' and SourceTableName LIKE '%{}%' and FileName LIKE '%{}%' and FileType LIKE '%{}%' and IsDeleted='0' and IsActive ='1'".format(source.rsplit('_', 1)[0], new_opu, docType, docFormat.lower())
        file_id = cursor.execute(file_id_query).fetchall()
        
    else: #THIS IS FOR PEDMS
        file_id_query = "SELECT ID FROM [dbo].[FileMetadata] meta INNER join [dbo].[PEDMS_OBID_Metadata] OBID ON meta.IDColumnValue = OBID.OBID and OBID.opu=meta.SourceTableName and meta.source ='{}' and OBID.opu='{}' and is_deleted='false' and OBID.Doc_Type_Name  ='{}' and meta.isactive = 1 and meta.isdeleted = 0 WHERE meta.source ='{}' and OBID.opu='{}' and OBID.Doc_Type_Name  ='{}' and meta.isactive = 1 and meta.isdeleted = 0".format(source.rsplit('_', 1)[0],opu,docType,source.rsplit('_', 1)[0],opu,docType)
        file_id = cursor.execute(file_id_query).fetchall()

    new_list = [name[0] for name in file_id]

    fileList_df = pd.DataFrame(new_list, columns=['ID'])
    folderPaths_df = pd.DataFrame(folder_paths,columns=['Path'])

    folderPaths_df['PathID'] = folderPaths_df['Path'].str.split('_',expand=True)[0].astype(int)

    filtered_folderPaths_df = folderPaths_df.merge(fileList_df, left_on='PathID', right_on='ID', how='inner')

    latest_folderPaths = filtered_folderPaths_df['Path'].tolist()

    return latest_folderPaths

def retrieve_file_names_from_sql_log(source,opu,docType):
    cursor = JDBCconnection.cursor()
    source_str = source.split('_')[0]

    if source_str == 'ECMS':
        docType = docType + '.'

    query = "SELECT SourceFileName FROM [dbo].[ingestion_log_image_extraction] WHERE Status = 'Succeeded' AND ExecutionType = 'Image Extraction' AND Source like '%{}%' AND OPU = '{}' AND SourceFileName LIKE '%{}%' ".format(source_str, opu,docType)
    result = cursor.execute(query).fetchall()

    result_list = [name[0] for name in result]
    df = pd.DataFrame({"File Name": result_list})
    df = df.drop_duplicates()

    new_list = df['File Name'].values.tolist()

    return new_list

In [0]:
## TO GET FILEPATH
path = "AZ_ADLS_PR_UNSTRUCTURED/{0}/{1}/V01/".format(source, opu)
process_fileSystem = 'process'
input_directory = "abfss://{0}@{1}.dfs.core.windows.net/AZ_ADLS_PR_UNSTRUCTURED/{2}/{3}/V01/".format(process_fileSystem,storage_acc,source,opu)
cursor = JDBCconnection.cursor()

documentFormat = ['PDF','DOC']
latest_folderPaths = []
for docFormat in documentFormat:
    incremental = is_incremental(cursor,opu,docType)
    print(incremental)
    print(docFormat)
    folder_paths=[]
    temp_folderPaths = []

    if incremental == '0':
    #proceed with normal load - Update Is_Incremental to 1 (so it will load incrementally in the future)        
        file_list = dbutils.fs.ls(input_directory)

        for folder in file_list:
            if docType.upper() in folder.name.upper() and docFormat.upper() in folder.name.upper():
                folder_paths.append(folder.name)

        if (len(folder_paths))>0:
            try:
                temp_folderPaths = path_validation(cursor, folder_paths, source, opu, docType,docFormat) #based on filemetadata
                print(len(temp_folderPaths))
                
            except Exception as e:
                print(e)

        if (len(temp_folderPaths))>0:
            extracted_files = retrieve_file_names_from_sql_log(source,opu,docType)
            print(len(extracted_files))
            new_list = [x for x in temp_folderPaths if x not in extracted_files]
            print(len(new_list))
            print(new_list)
            latest_folderPaths.append(new_list)
        else:
            print('There is no related documents to be executed!')

    else:
        file_list = get_blob(process_fileSystem, path, docType,docFormat)
        try:
            latest_time = latest_runtime(cursor,opu).timestamp()
        except:
            latest_time = int(datetime.fromtimestamp(0).timestamp())
        file_list_modified_date = []

        for i in file_list:
            storage_account_url="{}://{}.dfs.core.windows.net".format("https",storage_acc)
            service_client = DataLakeServiceClient(account_url=storage_account_url, credential=token_credential)
            directory_client = service_client.get_directory_client(process_fileSystem, i)
            directory_properties = directory_client.get_directory_properties()
            if directory_properties['last_modified'].timestamp() > latest_time:
                paths = os.path.basename(directory_properties['name'])
                folder_paths.append(paths)

        if len(folder_paths)>0:
            try:
                temp_folderPaths = path_validation(cursor, folder_paths, source, opu, docType, docFormat)
                print(len(temp_folderPaths))
                latest_folderPaths.append(temp_folderPaths)
            except Exception as e:
                print(e)
        else:
            print('There is no related documents to be executed')

In [0]:
class DocumentCracker():
    def __init__(self, source_client, target_adls_container):
        self.source_client = source_client
        self.target_adls_container = target_adls_container
        self.file_properties = source_client.get_file_properties()
        self.file_path = source_client.get_file_properties()['name']
        self.file_type = self.file_path.rsplit('.',1)[1].lower()
        self.extracted_images_hash = []
        self.images_extracted = 0
        self.images_extracted_failed = 0
        self.error_description = ''
        self.cursor = JDBCconnection.cursor()
        self.filemetadata_details = self.get_file_metadata_details(self.file_properties['metadata']['file_metadata_id'])
        
    def get_image_format(self, image_type):
        return {
            "jpeg": drawing.imaging.ImageFormat.jpeg,
            "emf": drawing.imaging.ImageFormat.emf,
            "bmp": drawing.imaging.ImageFormat.bmp,
            "png": drawing.imaging.ImageFormat.png,
            "wmf": drawing.imaging.ImageFormat.wmf,
            "gif": drawing.imaging.ImageFormat.gif,
        }.get(image_type, drawing.imaging.ImageFormat.jpeg)
    
    def log_start(self, log_details):
        try:
            self.cursor.execute("INSERT INTO [dbo].[ingestion_log_image_extraction](Source, SourceFileName, PipelineRunId, Status, ExecutionStartTime, OPU, ExecutionType) VALUES (?, ?, ?, 'In Progress', getdate(), ?, 'Image Extraction');", log_details['source'], log_details['source_file_name'], log_details['pipeline_run_id'],opu)
            log_id = self.cursor.execute("SELECT IDENT_CURRENT('[dbo].[ingestion_log_image_extraction]');").fetchone()[0]
            JDBCconnection.commit()
            return log_id
        except:
            print(str(traceback.format_exc()))
            return 'failed'
        
    def log_end(self, log_details):
        try:
            self.cursor.execute("UPDATE [dbo].[ingestion_log_image_extraction] set Status = ?, ErrorDescription = ?, ExecutionEndTime = getdate(), ImagesExtracted = ?, ImagesExtractFailed = ?, OPU = ? where ID = ?;", log_details['status'], log_details['error_description'], log_details['image_extracted'], log_details['image_extracted_failed'],log_details['opu'], log_details['log_id'])
            JDBCconnection.commit()
            return 'succeeded' 
        except:
            print(str(traceback.format_exc()))
            return 'failed'
        
    def get_target_file_client(self, file_path_name):
        return DataLakeFileClient(
            account_url=account_url, 
            file_system_name = self.target_adls_container, 
            file_path=file_path_name, 
            credential=token_credential
        )
        
    def get_file_metadata_details(self, filemetadata_id):
        try:
            query = "SELECT Source, SourceDatabase, SourceTableName FROM FileMetadata WHERE ID = '{}'".format(filemetadata_id)
            result = cursor.execute(query).fetchone()
            metadata_properties = {
                'Source': result[0],
                'SourceDatabase': result[1],
                'SourceTableName': result[2]
            }
            return metadata_properties
        except:
            print('metadata not found for: ' + self.file_path)
            #raise SystemExit
    
    def update_filemetadata(self, **details):        
        query = "SET NOCOUNT ON \
            DECLARE @ChangeResult TABLE (Id INTEGER)\
            MERGE INTO [dbo].[FileMetadata] AS TargetTbl USING (VALUES\
            ('{SourceDatabase}', '{SourceTableName}', 'FileMetadata_ID', '{IDColumnValue}', '{FileName}', '{FileType}', '{BlobPath}', '{Source}')\
            ) AS SourceTbl (SourceDatabase, SourceTableName, IDColumnName, IDColumnValue, FileName, FileType, BlobPath, Source)\
            ON (TargetTbl.Source = SourceTbl.Source AND TargetTbl.IDColumnValue = SourceTbl.IDColumnValue AND substring(TargetTbl.FileName, charindex('_', TargetTbl.FileName) + 1 , len(TargetTbl.FileName)) = SourceTbl.FileName AND TargetTbl.IsActive = 1)\
            WHEN MATCHED THEN\
            UPDATE SET\
            FileName = concat(TargetTbl.ID, '_', SourceTbl.FileName), FileType = SourceTbl.FileType, BlobPath = concat(SourceTbl.BlobPath,TargetTbl.ID,'_',SourceTbl.FileName), ModifiedDate = getdate(), IsDeleted = 0\
            WHEN NOT MATCHED BY TARGET THEN\
            INSERT (SourceDatabase,SourceTableName,IDColumnName,IDColumnValue,FileType,IsActive,CreatedDate, FileName, BlobPath, Source, IsDeleted) values\
            (SourceTbl.SourceDatabase,SourceTbl.SourceTableName,SourceTbl.IDColumnName,SourceTbl.IDColumnValue,SourceTbl.FileType, 1, getdate() ,(select concat((select IDENT_CURRENT('[dbo].[FileMetadata]')),'_', SourceTbl.FileName)) ,(select concat(SourceTbl.BlobPath, (select IDENT_CURRENT('[dbo].[FileMetadata]')),'_', SourceTbl.FileName)), SourceTbl.Source, 0)\
            OUTPUT inserted.Id INTO @ChangeResult;\
            SELECT * FROM @ChangeResult"\
            .format(SourceDatabase=details['SourceDatabase'],
            SourceTableName=details['SourceTableName'],
            IDColumnValue=details['IDColumnValue'],
            FileName=details['FileName'],
            FileType=details['FileType'],
            BlobPath=details['BlobPath'],
            Source=details['Source'])
        metadata_id = self.cursor.execute(query).fetchone()[0]
        JDBCconnection.commit()
        return str(metadata_id)

    def validate_image(self, image_bytes):
        if hashlib.md5(image_bytes).digest() not in self.extracted_images_hash and len(image_bytes) > 0 and magic.from_buffer(bytes(image_bytes), mime=True).split('/',1)[0] == 'image':
            return True
        else:
            return False

    def crack_document(self):
        # Log Progress
        log_details = {
            "source": self.filemetadata_details['Source'], 
            "source_file_name":os.path.basename(self.file_path),
            "pipeline_run_id": pipelineRunID
        }
        log_id = self.log_start(log_details)
        
        # Extract images based on file type 
        if self.file_type in ['pdf']:
            self.crack_pdf()
        elif self.file_type in ['docx', 'doc']:
            self.crack_docx()
        elif self.file_type in ['pptx', 'ppt']:
            self.crack_pptx()
            
        if self.images_extracted_failed < 1:
            log_details = {
                "status": 'Succeeded', 
                "error_description":None,
                "image_extracted": self.images_extracted,
                "image_extracted_failed": self.images_extracted_failed,
                "log_id": log_id,
                "opu": opu
            }
        else:
            log_details = {
                "status": 'Failed', 
                "error_description": self.error_description if self.images_extracted_failed < 1 else ("Error extracting certain images. " + self.error_description),
                "image_extracted": self.images_extracted,
                "image_extracted_failed": self.images_extracted_failed,
                "log_id": log_id,
                "opu": opu
            }
        self.log_end(log_details)   
            
        # Close Connection
        self.cursor.close()

    def crack_pdf(self):
        generated_count=0
        try:
            ## Oscar's detection of Scanned/Generated Documents
            raw_content = fileClient_source.download_file().readall() #Download file into bytes

            result,numpage,ans= PDFPageCheck(raw_content)
            
            #identify the pdf file is generated or scanned  
            isGenerated = isGeneratedPDF(result,numpage,ans)

            if(isGenerated is True):
                print('\n' + self.file_path+'- Generated file : '+ str(isGenerated)) 
                pdf_file = fitz.open(stream=raw_content, filetype="pdf")

                ## Oscar's detection of Scanned/Generated Documents
                img_index = 1
                filemetadata_id = self.file_properties['metadata']['file_metadata_id']
                
                caption=[]

                # iterating through each page in the pdf
                for current_page_index in range(len(pdf_file)):
                    img_count=0
                    caption = getPageText(pdf_file,current_page_index) #get caption of images in the PDF file
                    imagelist=[]
                    # iterating through each image in every page of PDF
                    for _ ,img in enumerate(pdf_file.get_page_images(current_page_index)):
                        xref = img[0]
                        image = fitz.Pixmap(pdf_file, xref)
                        image_filename = "{}_extracted image_{}.png".format(filemetadata_id, img_index)
                        #image = Image.open(image_file)
                        image_width = image.width
                        image_height = image.height

                        # if it is CMYK: convert to RGB first
                        if image.n >= 5:   #5                  
                            image = fitz.Pixmap(fitz.csRGB, image)

                        image_bytes = image.tobytes()
                        
                        if self.validate_image(image_bytes) and image_width >= 380 and image_height >= 230:
                            try:
                                # Update Metadata 
                                new_metadata_id = self.update_filemetadata(
                                    SourceDatabase=self.filemetadata_details['SourceDatabase'],
                                    SourceTableName=self.filemetadata_details['SourceTableName'],
                                    IDColumnValue=filemetadata_id,
                                    FileName=os.path.basename(image_filename),
                                    FileType= '.' + image_filename.rsplit('.',1)[1],
                                    BlobPath=os.path.dirname(self.file_path) + '/',
                                    Source=self.filemetadata_details['Source']
                                )
                               
                                image_filename = ("{}/{}_".format(os.path.dirname(self.file_path), new_metadata_id) + image_filename).replace('//', '/')
                                print(image_filename)
                                md5_hash = hashlib.md5(image_bytes).digest()
                                self.extracted_images_hash.append(md5_hash)
                                img_count += 1
                                print("\nSuccessfully saved png:", image_filename)
                                
                                # Upload to ADLS 
                                target_client = self.get_target_file_client(image_filename)
                                metadata = {'file_metadata_id':new_metadata_id, 'classification':self.file_properties['metadata']['classification']}
                                
                                # Get content type 
                                processedContentType = magic.from_buffer(image_bytes, mime=True)
                                
                                # Specify ContentType for file 
                                fileContent = ContentSettings(content_type=processedContentType, content_md5=md5_hash)
                                target_client.upload_data(image_bytes, overwrite=True, metadata=metadata, content_settings = fileContent)

                                #test
                                #filename = getImageFileMetadataId (metadata , cursor , Source) 
                                #stored the image data into imagelist
                                #imagelist.append([metadata,filename,len(pdf_file)+1,blobPath+metadata+image_filename,isDeleted])
                                isDeleted = 'False'
                                #imagelist.append([new_metadata_id,self.filemetadata_details['SourceTableName'],len(pdf_file)+1,image_filename,isDeleted])
                                imagelist.append([new_metadata_id,opu,len(pdf_file)+1,image_filename,isDeleted])
                                
                                self.images_extracted = img_index
                                img_index += 1
                                del image_bytes
                        
                            except Exception as e:
                                self.images_extracted_failed += 1 
                                print(str(traceback.format_exc()))
                        else:
                            #self.images_extracted += 1
                            del image_bytes 

                    #Include caption generation and will be updated to DB
                    if(img_count>0 and len(caption)>0):
                        updateCaption(caption, img_count, imagelist)

            else:
                print('\n' + self.file_path+'- Generated file : '+ str(isGenerated)) 
                #print('\n' + self.file_path + " is a scanned document")

        except:
            self.error_description += str(traceback.format_exc()) + ' || ' + self.error_description
            print(self.error_description)
            
    def crack_docx(self):
        try:
            exclude_aspose_image = b'\xc0<\xd7r\xc9\x16\xd9<\xb3g\xd8\xfe\xdc\x98\xb0\xec'
            print("\nProcessing DOC/DOCX", self.file_path)

            # load the Word document
            streamdownloader=self.source_client.download_file()
            streamdownloader_bytes = streamdownloader.readall()
            doc = aw.Document(BytesIO(streamdownloader_bytes))
            filemetadata_id = self.file_properties['metadata']['file_metadata_id']
            
            # create a list of the first 5 pages
            # pages = [document.sections[0].header, document.sections[0].footer, document.sections[0].first_page_header, document.sections[0].first_page_footer, document.sections[0].page]
            #print(pages)

            # retrieve all shapes
            shapes = doc.get_child_nodes(aw.NodeType.SHAPE, True)
            imageIndex = 1

            # loop through shapes
            for shape in shapes :
                shape = shape.as_shape()

                if shape.has_image:
                    shape_bytes = shape.image_data.image_bytes
                    shape_hash = hashlib.md5(shape_bytes).digest()

                    shape_width = shape.width
                    shape_height = shape.height
                    #print(shape_width, shape_height)

                    if self.validate_image(shape_bytes) and shape_hash != exclude_aspose_image and shape_width >380 and shape_height > 230:
                        try:
                            # set image file's name
                            image_filename = "{}_extracted image_{}.png".format(filemetadata_id, imageIndex)

                            # Update Metadata 
                            new_metadata_id = self.update_filemetadata(
                                SourceDatabase=self.filemetadata_details['SourceDatabase'],
                                SourceTableName=self.filemetadata_details['SourceTableName'],
                                IDColumnValue=filemetadata_id,
                                FileName=os.path.basename(image_filename),
                                FileType= '.' + image_filename.rsplit('.',1)[1],
                                BlobPath=os.path.dirname(self.file_path) + '/',
                                Source=self.filemetadata_details['Source']
                            )
                            
                            #image_filename = "{}/testing/{}_".format(os.path.dirname(self.file_path), new_metadata_id) + image_filename
                            image_filename = ("{}/{}_".format(os.path.dirname(self.file_path), new_metadata_id) + image_filename).replace('//', '/')

                            # save image
                            self.extracted_images_hash.append(shape_hash)
                            print("Successfully saved png:", image_filename)
                            
                            target_client = self.get_target_file_client(image_filename)
                            # Get content type 
                            processedContentType = magic.from_buffer(bytes(shape_bytes), mime=True)
                            # Specify ContentType for file 
                            fileContent = ContentSettings(content_type=processedContentType, content_md5=shape_hash)
                            metadata = {'file_metadata_id':new_metadata_id, 'classification':self.file_properties['metadata']['classification']}
                            target_client.upload_data(bytes(shape_bytes), overwrite=True, metadata=metadata, content_settings = fileContent)
                            self.images_extracted = imageIndex
                            imageIndex += 1
                            
                        except Exception as e:
                            self.images_extracted_failed += 1 
                            print(traceback.format_exc())
                        else:
                            del image_bytes
        except:
            self.error_description += str(traceback.format_exc()) + ' || ' + self.error_description
            
    def crack_pptx(self):
        try:
            print("\nProcessing PPT/PPTX", self.file_path)
            imageIndex=0
            # load presentation
            filemetadata_id = self.file_properties['metadata']['file_metadata_id']
            streamdownloader=self.source_client.download_file()
            streamdownloader_bytes = streamdownloader.readall()
            with slides.Presentation(BytesIO(streamdownloader_bytes)) as pres:

                # loop through images
                for image in pres.images:
                    image_type = image.content_type.split("/")[1]
                    image_bytes = image.binary_data
                    
                    #image_width = image.width
                    #print("width: ", image_width)
                    image_width = image.width
                    image_height = image.height

                    if self.validate_image(image_bytes) and image_width > 380 and image_height >230 and image_type.lower() in ['jpg', 'jpeg', 'png', 'gif', 'bmp']:
                        try:
                            image_filename = "{}_extracted image_{}.{}".format(filemetadata_id, imageIndex, image_type)
                            image_format = self.get_image_format(image_type)
                            md5_hash = hashlib.md5(image_bytes).digest()

                            # Update Metadata 
                            new_metadata_id = self.update_filemetadata(
                                SourceDatabase=self.filemetadata_details['SourceDatabase'],
                                SourceTableName=self.filemetadata_details['SourceTableName'],
                                IDColumnValue=filemetadata_id,
                                FileName=os.path.basename(image_filename),
                                FileType= '.' + image_filename.rsplit('.',1)[1],
                                BlobPath=os.path.dirname(self.file_path) + '/',
                                Source=self.filemetadata_details['Source']
                            )
                            
                            image_filename = "{}/{}_".format(os.path.dirname(self.file_path), new_metadata_id) + image_filename

                            # save image
                            self.extracted_images_hash.append(md5_hash)
                            imageIndex += 1

                            target_client = self.get_target_file_client(image_filename)
                            # Get content type 
                            processedContentType = magic.from_buffer(bytes(image_bytes), mime=True)
                            # Specify ContentType for file 
                            fileContent = ContentSettings(content_type=processedContentType, content_md5=md5_hash)
                            metadata = {'file_metadata_id':new_metadata_id, 'classification':self.file_properties['metadata']['classification']}
                            target_client.upload_data(bytes(image_bytes), overwrite=True, metadata=metadata, content_settings = fileContent)
                            print("Successfully saved png:", image_filename)
                        except Exception as e:
                            self.images_extracted_failed += 1 
                            print(traceback.format_exc())
                        else:
                            slf.images_extracted += 1 
        except:
            self.error_description += str(traceback.format_exc()) + ' || ' + self.error_description

In [0]:
path = '/AZ_ADLS_PR_UNSTRUCTURED/{0}/{1}/V01/'.format(source,opu) #These are for folder_path loop

latest_folderPaths = [item for sublist in latest_folderPaths for item in (sublist if isinstance(sublist, list) else sublist)]
cursor = JDBCconnection.cursor()

if (len(latest_folderPaths)) > 0:
    for path_number in range(len(latest_folderPaths)):
        full_path = path+str(latest_folderPaths[path_number])

        fileClient_source = DataLakeFileClient(
            account_url=account_url, 
            file_system_name ='process', 
            file_path=full_path,
            credential=token_credential
        )

        try: 
            pipelineRunID 
        except:
            pipelineRunID = '00000000-0000-0000-0000-000000000000'

        if fileClient_source.exists():
            document_cracker = DocumentCracker(fileClient_source, 'process')
            try:
                document_cracker.crack_document()
            except:
                continue
        else:
            print("File not exists")

    update_incremental(cursor,opu,docType)

else:
    update_incremental(cursor,opu,docType)
    print('There are no documents to be executed')